In [21]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

In [22]:
df = pd.read_csv("/data/BBox_List_2017.csv")

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Image   984 non-null    object 
 1   Label   984 non-null    object 
 2   x       984 non-null    float64
 3   y       984 non-null    float64
 4   w       984 non-null    float64
 5   h       984 non-null    float64
dtypes: float64(4), object(2)
memory usage: 46.2+ KB


In [24]:
def function(data):
    X = data.drop(["Label"], axis=1).values
    y = data["Label"].values
    
    ss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=0)
    ss.get_n_splits(X, y)
    for i, (train_index, test_index) in enumerate(ss.split(X, y)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
    x_cols = df.drop(["Label"], axis=1).columns.to_list()
    y_cols = df[["Label"]].columns.to_list()
    
    train = pd.DataFrame(data=X_train, columns=x_cols)
    test = pd.DataFrame(data=X_test, columns=x_cols)
    
    train[y_cols[0]] = y_train
    test[y_cols[0]] = y_test
    
    return train,test

In [26]:
train,test = function(df)

In [27]:
train.to_csv("/data/train.csv", index=False)
test.to_csv("/data/test.csv", index=False)

In [28]:
set(train.Label.to_list())

{'Atelectasis',
 'Cardiomegaly',
 'Effusion',
 'Infiltrate',
 'Mass',
 'Nodule',
 'Pneumonia',
 'Pneumothorax'}